In [1]:
import cv2
from picamera2 import Picamera2
from ultralytics import YOLO
import matplotlib.pyplot as plt
import pickle
import os
import numpy as np
import time
import serial
os.environ["DISPLAY"] = ":0"

In [2]:

model_path = "/home/pi/EZLift/src/raspberry_pi/custom_palletsonly_50epochs_edgetpu.tflite"

model = YOLO(model_path, task='detect')

In [3]:
# import time
# import cv2

# picam2 = Picamera2()
# picam2.preview_configuration.main.size = (3280, 2464)
# picam2.preview_configuration.main.format = "RGB888"
# picam2.preview_configuration.align()
# picam2.configure("preview")
# picam2.start()
# picam2.start(show_preview=False)
# frame = picam2.capture_array()

# with open("frame.pkl", "wb") as f:
#     pickle.dump(frame, f)

In [4]:
def _get_center_error(bbox, img):

    bbox_center = (bbox[0] + bbox[2]) / 2
    img_center = img.shape[1] / 2
    error = img_center - bbox_center
    return error

In [5]:
picam2 = Picamera2()
picam2.preview_configuration.main.size = (3280, 2464)
picam2.preview_configuration.main.format = "RGB888"
picam2.preview_configuration.align()
picam2.configure("preview")
picam2.start()
picam2.start(show_preview=False)
counter = 0
serial_port = serial.Serial("/dev/ttyACM0", 9600)

while True:
    start_time = time.time()
    
    # Capture frame-by-frame
    frame = picam2.capture_array()

    # Run YOLO inference on the frame with imgsz=256
    frame = cv2.resize(frame, (256, 256))
    results = model.predict(frame, device="tpu:0", imgsz=256)
        
    error = _get_center_error(results[0].boxes.xyxy[0], frame)
    # Visualize the results on the frame
    annotated_frame = results[0].plot()

    # Calculate FPS
    end_time = time.time()
    fps = 1.0 / (end_time - start_time)
    
    # Overlay the FPS on the annotated frame
    cv2.putText(annotated_frame, f"FPS: {error:.2f}", (10, 30), 
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
    
    # Display the resulting frame
    cv2.imshow("Camera", annotated_frame)
    cv2.waitKey(1)

    # Print FPS to the console (optional)
    print(f"FPS: {fps:.2f}")

[4:11:43.631618060] [9127]  INFO Camera camera_manager.cpp:327 libcamera v0.4.0+53-29156679
[4:11:43.673251963] [9158] ERROR V4L2 v4l2_device.cpp:390 'imx219 10-0010': Unable to set controls: Device or resource busy
[4:11:43.693094580] [9158]  WARN RPiSdn sdn.cpp:40 Using legacy SDN tuning - please consider moving SDN inside rpi.denoise
[4:11:43.695471465] [9158]  WARN RPI vc4.cpp:393 Mismatch between Unicam and CamHelper for embedded data usage!
[4:11:43.696448114] [9158]  INFO RPI vc4.cpp:447 Registered camera /base/soc/i2c0mux/i2c@1/imx219@10 to Unicam device /dev/media3 and ISP device /dev/media0
[4:11:43.696538002] [9158]  INFO RPI pipeline_base.cpp:1121 Using configuration file '/usr/share/libcamera/pipeline/rpi/vc4/rpi_apps.yaml'
Camera __init__ sequence did not complete.
[4:11:43.707501310] [9127]  INFO Camera camera.cpp:1008 Pipeline handler in use by another process


RuntimeError: Camera __init__ sequence did not complete.

In [ ]:
# input_path = r"/home/pi/EZLift/src/raspberry_pi/notebooks/test_imgs/istockphoto-1318934972-612x612.jpg"
# img = cv2.imread(input_path)
# img = cv2.resize(img, (256, 256))
# results = model.predict(img, device="tpu:0", imgsz=256)

In [ ]:
# boxes = results[0].boxes
# x1, y1, x2, y2 = boxes[0].xyxy[0].cpu().numpy()
# x1, y1, x2, y2 = map(int, (x1, y1, x2, y2))
# conf = boxes[0].conf[0].cpu().numpy()
# cls_id = int(boxes[0].cls[0].cpu().numpy())

# bb_crop = results[0].orig_img[y1:y2, x1:x2]